In [1]:
# Import libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 
from pyproj import Proj, transform
import folium

In [2]:
# Read the dataset
carr_data = '../_data/GMO_CARREFOUR.csv'
carr_df = pd.read_csv(carr_data, sep=';', encoding='latin-1')

In [16]:
carr_df.dtypes

ID_GM_CARREFOUR    float64
TYPE                object
COMMENTAIRE         object
DATE_MAJ            object
RAYON              float64
E                  float64
N                  float64
dtype: object

In [3]:
carr_df.TYPE.value_counts()

Autres carrefours                      5852
Impasse                                1272
Signalisation lumineuse                 379
Changement voie                         216
Giratoire                               198
Frontière                               119
Stop toutes directions                   32
Giratoire à signalisation lumineuse       7
Name: TYPE, dtype: int64

In [18]:
carr_df.DATE_MAJ.value_counts().sort_index()

2008-04-18 16:20:43           815
2008-04-18 16:20:44           112
2008-04-18 16:20:51           930
2008-04-18 16:21:02           915
2008-04-18 16:21:11           917
2008-04-18 16:21:21           930
2008-04-18 16:21:32           494
2008-04-18 16:21:33.000001    419
2008-04-18 16:21:46           936
2008-04-18 16:22:01           751
2008-04-18 16:24:25.000001      5
2008-04-22 08:06:26             1
2008-04-22 14:22:59             1
2008-04-29 11:00:50             1
2008-05-21 09:00:06             1
2008-05-21 09:01:17             1
2008-05-21 09:03:20             1
2008-05-27 09:02:49            13
2008-05-29 08:36:39             1
2008-05-29 08:37:07.000001      1
2008-05-29 08:37:28             1
2008-05-29 08:38:09             1
2008-05-29 08:38:35             1
2008-05-29 08:38:50             1
2008-06-11 14:13:32             1
2008-06-11 14:16:09             1
2008-06-12 09:31:43             1
2008-06-12 09:34:19             1
2008-06-12 09:35:36             1
2008-06-12 09:

In [4]:
# projection definition
p1 = Proj(init='epsg:2056')
p2 = Proj(init='epsg:4326')

# Helper functions
def coord_proj(carr_df,i, p1, p2):
    x1 = carr_df['E'].loc[i]
    y1 = carr_df['N'].loc[i]
    x2, y2 = transform(p1,p2,x1,y1)
    carr_df['E'].set_value(i, x2)
    carr_df['N'].set_value(i, y2)
    return carr_df

# Project data
for i in range(0, len(carr_df['E'])-1):
    carr_df = coord_proj(carr_df,i, p1, p2)
# Delete unuseful columns
#del acc_df['N']
#del acc_df['E']

In [5]:
carr_df.head(3)

,ID_GM_CARREFOUR,TYPE,COMMENTAIRE,DATE_MAJ,RAYON,E,N
0,227920.0,Autres carrefours,NaN,2008-04-18 16:20:43,NaN,5.997844,46.158031
1,226972.0,Autres carrefours,NaN,2008-04-18 16:20:43,NaN,6.117844,46.192797
2,226973.0,Autres carrefours,NaN,2008-04-18 16:20:43,NaN,6.118345,46.193008


In [6]:
giratoire_df = carr_df[carr_df.TYPE=='Giratoire']
giratoire_df.head()

,ID_GM_CARREFOUR,TYPE,COMMENTAIRE,DATE_MAJ,RAYON,E,N
47,225343.0,Giratoire,NaN,2008-10-15 08:43:14,14.0,6.132505,46.267669
225,228208.0,Giratoire,NaN,2008-10-13 16:37:15,13.0,6.153596,46.292103
241,225549.0,Giratoire,NaN,2008-10-16 15:17:59,12.0,6.120854,46.238227
385,226606.0,Giratoire,NaN,2008-10-14 08:56:20,13.0,6.075616,46.232344
589,223968.0,Giratoire,NaN,2008-10-14 08:24:10,12.0,6.203407,46.203628


In [10]:
# Create the map visualization
Geneve_coord = [46.2004013, 6.1531163]
m = folium.Map(location=Geneve_coord,
               tiles='OpenStreetMap',
               zoom_start=10)
for i in giratoire_df.index:
    folium.Marker([giratoire_df.N.loc[i], giratoire_df.E.loc[i]] , popup="Giratoire, %d"%i).add_to(m)
m

Giratoire build (finished) in 2014: possible case study for confronting carrefour VS giratoire
https://www.ge.ch/construction/pdf/chantiers/Communique_de_presse_Route_de_Thonon-giratoire_FINAL_28_06_2013.pdf

In [14]:
giratoire_df.loc[7741]

ID_GM_CARREFOUR                 226855
TYPE                         Giratoire
COMMENTAIRE                        NaN
DATE_MAJ           2016-02-19 14:07:06
RAYON                               15
E                              6.20622
N                              46.2453
Name: 7741, dtype: object